In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import KFold

os.chdir("/g/data/jr19/rh2942/text-empathy/")
from utils.utils import plot, set_all_seeds
from utils.kfold import KFoldDataModule, KFoldTrainer

In [2]:
checkpoint = 'roberta-base'
task = ['empathy', 'wrong_empathy']
# feature_to_tokenise=['demographic_essay', 'article']
# feature_to_tokenise=['demographic', 'essay']
feature_to_tokenise=['demographic_essay']
seed = 0
anno_diff_range = np.arange(0, 6.5, 0.5)

################# COMBINED TRAIN FILE ##############
# train_file = './data/WS22-WS23-train-gpt.tsv'
# train_file = './data/WS22-WS23-sep-from-aug-train-gpt.tsv'
# train_file = './data/essay-train-ws22-ws23.tsv'
# train_file = './data/PREPROCESSED-WS22-WS23-train.tsv'

## Augmented
# train_file = './data/COMBINED-PREPROCESSED-PARAPHRASED-WS22-WS23-train.tsv'
# train_file = './data/WS22-WS23-augmented-train-gpt.tsv'

################# WASSA 2022 ####################
# dev_file = './data/PREPROCESSED-WS22-dev.tsv'
# dev_label_file = './data/WASSA22/goldstandard_dev_2022.tsv'
# test_file = './data/PREPROCESSED-WS22-test.tsv'

##### GPT annotation
# train_file = './data/WS22-train-gpt.tsv'
# dev_file = './data/WS22-dev-gpt.tsv'
# dev_label_file = './data/WS22-dev-gpt.tsv'

#################### WASSA 2021 ###################
filename = './data/WS21-10-percent-removed.tsv'


################# WASSA 2023 ####################
# dev_file = './data/PREPROCESSED-WS23-dev.tsv'
# dev_label_crowd = './data/WASSA23/goldstandard_dev.tsv'
# test_file = './data/PREPROCESSED-WS23-test.tsv'

##### GPT annotation
# train_file = './data/WS23-train-gpt.tsv'
# dev_file = './data/WS23-dev-gpt.tsv'
# dev_label_gpt = './data/WS23-dev-gpt.tsv'

In [3]:
kfold_results = pd.DataFrame()

set_all_seeds(seed)

data_module = KFoldDataModule(
    task=task,
    checkpoint=checkpoint,
    batch_size=16,
    feature_to_tokenise=feature_to_tokenise,
    seed=seed
)

data = data_module.get_data(file=filename)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

for fold, (train_idx, test_idx) in enumerate(kfold.split(data)):
    print('Fold:', fold)
    
    train_loader = data_module.kfold_dataloader(
        file=filename, idx=train_idx
    )
    dev_loader = data_module.kfold_dataloader(
        file=filename, idx=test_idx
    )

    for anno_diff in anno_diff_range:
        trainer = KFoldTrainer(
            task=task,
            checkpoint=checkpoint,
            lr=1e-5,
            n_epochs=10,
            train_loader=train_loader,
            dev_loader=dev_loader,
            dev_label_gpt=filename,
            dev_label_crowd=None,
            device_id=0,
            anno_diff=anno_diff,
            mode=0 # -1: crowd, 1: gpt, 0: crowd-gpt
        )
    
        val_pearson_r = trainer.fit()
    
        # save as seed in index and anno_diff in columns
        print(f'\n----Pearson r: {val_pearson_r}----\n')
        kfold_results.loc[fold, anno_diff] = val_pearson_r

    # # Saving in each fold to be cautious
    kfold_results.to_csv('./ws21-kfold_results_anno_diff.tsv', sep='\t')

# # Saving at the end
kfold_results.to_csv('./ws21-kfold_results_anno_diff.tsv', sep='\t')

Map:   0%|          | 0/1669 [00:00<?, ? examples/s]

Fold: 0


Map:   0%|          | 0/1669 [00:00<?, ? examples/s]

Map:   0%|          | 0/1669 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch: 1
Train loss: 12.330717621965611
Pearson r: 0.892
Validation loss: 0.4027002453804016
Best Pearson r: 0.892

Epoch: 2
Train loss: 3.4153337656183447
Pearson r: 0.892
Validation loss: 0.3421345353126526
Best Pearson r: 0.892

Epoch: 3
Train loss: 3.300850033760071
Pearson r: 0.894
Validation loss: 0.3353433609008789
Best Pearson r: 0.894

Epoch: 4
Train loss: 2.9449580882457975
Pearson r: 0.9
Validation loss: 0.3236311674118042
Best Pearson r: 0.9

Epoch: 5
Train loss: 2.657196088040129
Pearson r: 0.904
Validation loss: 0.3204941153526306
Best Pearson r: 0.904

Epoch: 6
Train loss: 2.2843999368079166
Pearson r: 0.899
Validation loss: 0.35251036286354065
Best Pearson r: 0.904

Epoch: 7
Train loss: 2.059312715175304
Pearson r: 0.886
Validation loss: 0.360188752412796

----Pearson r: 0.904----



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1
Train loss: 10.855143582567255
Pearson r: 0.877
Validation loss: 0.35390669107437134
Best Pearson r: 0.877

Epoch: 2
Train loss: 3.3599959865529487
Pearson r: 0.878
Validation loss: 0.35503876209259033
Best Pearson r: 0.878

Epoch: 3
Train loss: 3.259401392429433
Pearson r: 0.886
Validation loss: 0.3231244385242462
Best Pearson r: 0.886

Epoch: 4
Train loss: 2.810347842409256
Pearson r: 0.882
Validation loss: 0.34774503111839294
Best Pearson r: 0.886

Epoch: 5
Train loss: 2.490340902450237
Pearson r: 0.347
Validation loss: 5.093901634216309
Best Pearson r: 0.886

Epoch: 6
Train loss: 2.2727647474471557
Pearson r: 0.739
Validation loss: 0.81618332862854

----Pearson r: 0.886----



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1
Train loss: 11.735700143144486
Pearson r: 0.877
Validation loss: 0.4014798104763031
Best Pearson r: 0.877

Epoch: 2
Train loss: 3.4793074524148984
Pearson r: 0.878
Validation loss: 0.35660630464553833
Best Pearson r: 0.878

Epoch: 3
Train loss: 3.3268684846289616
Pearson r: 0.879
Validation loss: 0.3522029519081116
Best Pearson r: 0.879

Epoch: 4
Train loss: 3.319319250735831
Pearson r: 0.879
Validation loss: 0.351287305355072
Best Pearson r: 0.879

Epoch: 5
Train loss: 3.2840554955157826
Pearson r: 0.882
Validation loss: 0.34931454062461853

----Pearson r: 0.882----

